In [ ]:
import geopandas as gpd
import pyarrow
import pandas as pd
import os

In [ ]:
target_2631 = pd.read_parquet('data/GPS.nosync/filter/Sonoma_2631_new', engine='pyarrow')

In [ ]:

def write_file(filename, info):
  text_file = open(filename, "w")
  text_file.write(info)
  text_file.close()

In [ ]:
len(target_2631)

In [ ]:
data = {'index': [-1], 'caid': [0], 'start_time' : [], 'end_time' : [], 'valid_count' : [], 'is_success' : [] }  
  
# Create DataFrame  
df_new = pd.DataFrame(data)

In [ ]:
df_new

In [ ]:
start_index = 0
start_id = target_2631.loc[start_index]['caid']
start_time = target_2631.loc[start_index]['la_datetime']
end_time = target_2631.loc[start_index]['la_datetime']
#data = {'index': [-1], 'caid': [start_id], 'start_time' : [start_time], 'end_time' : [end_time], 'valid_count' : [0], 'is_success' : [False] }  
df_new = pd.DataFrame(columns=['index', 'caid', 'start_time', 'end_time', 'valid_count', 'is_success'])
# Create DataFrame  
df_new = pd.DataFrame(data)
print("start index: %d, caid: %s" %(0, start_id))
end = 4724838
end_tmp = 500
count = 3077
valid_count = 0
is_first = True
for index in range(0, end):
  data_id = target_2631.loc[index]['caid']
  single_data = target_2631.loc[index]
  is_data_valid = (single_data['latitude'] < 38.85) & (single_data['latitude'] > 38.099) & (single_data['longitude'] < -122.35) & (single_data['longitude'] > -123.55)
  if is_data_valid:
    valid_count+=1
  if data_id != start_id:
    # print("end index: %d, caid: %s" %(index - 1, target_2631.loc[index - 1]['caid']))
    print("total points: %d" %(index - start_index))
    if valid_count >= 9:
       df_new.loc[count] = {'index': count, 'caid': target_2631.loc[index - 1]['caid'], 'start_time' : start_time, 'end_time' : target_2631.loc[index - 1]['la_datetime'], 'valid_count' : valid_count, 'is_success' : False }
    is_first = True
    count+=1
    start_index = index
    start_id = data_id
    start_time = target_2631.loc[index]['la_datetime']
    valid_count = 0
  #df.loc[len(df.index)] = [float(lat),float(lon)]
  # if is_data_valid:
  #   if is_first:
  #     is_first = False
  #     if valid_count >= 49:
  #       df_new.loc[count] = {'index': count, 'caid': target_2631.loc[index - 1]['caid'], 'start_time' : start_time, 'end_time' : target_2631.loc[index - 1]['la_datetime'], 'valid_count' : valid_count, 'is_success' : False }
  #     valid_count = 0
  #   else:
  #       if valid_count >= 49:
  #         df_new.loc[count] = {'index': count, 'caid': target_2631.loc[index - 1]['caid'], 'start_time' : start_time, 'end_time' : target_2631.loc[index - 1]['la_datetime'], 'valid_count' : valid_count, 'is_success' : False }
  #       valid_count = 0

    # print("start index: %d, caid: %s" %(index, data_id))
#print("end index: %d, caid: %s" %(end, target_2631.loc[end]['caid']))

In [ ]:
df_new.to_csv('out2.csv')

In [ ]:
t1 = pd.Timestamp(year = 2019,  month = 10, day = 27,
           hour = 3, minute = 20, second = 18, tz = 'US/Pacific')

df_new.loc[3078]['start_time']



In [ ]:
t1 > df_new.loc[3078]['start_time']

In [ ]:
start_index = 0
start_id = target_2631.loc[start_index]['caid']
#print("start index: %d, caid: %s" %(0, start_id))
end = 200000
count = 0
for index in range(0, end):
  data_id = target_2631.loc[index]['caid']
  if data_id != start_id:
    #print("end index: %d, caid: %s" %(index - 1, target_2631.loc[index - 1]['caid']))
    #print("total points: %d" %(index - start_index))
    print("%d start: %d, end: %d, total: %d, caid: %s" %(count, start_index, index - 1, (index - start_index), data_id))
    count+=1
    start_index = index
    start_id = data_id
    
    #print("start index: %d, caid: %s" %(index, data_id))
#print("end index: %d, caid: %s" %(end, target_2631.loc[end]['caid']))



In [ ]:

res = 'LINESTRING('
df = pd.DataFrame(
    {'Latitude': [float(target_2631.loc[72]['latitude'])],
     'Longitude': [float(target_2631.loc[72]['longitude'])]})

#for index in range_:
start_index = 72
end_index = 266
is_first = True
for index in range(start_index,end_index):
  
  single_data = target_2631.loc[index]
  if ((single_data['latitude'] < 38.85) & (single_data['latitude'] > 38.099) & (single_data['longitude'] < -122.35) & (single_data['longitude'] > -123.55)):
    lat = float(single_data['latitude'])
    lon = float(single_data['longitude'])
    df.loc[len(df.index)] = [float(lat),float(lon)]
    if is_first:
      is_first = False
      res+= (str(lon) + ' ' + str(lat))
    else:
      res+= (',' + str(lon) + ' ' + str(lat))
res+=')'
df = df[0:-1]
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
write_file('data/GPS.nosync/output/' + target_2631.loc[start_index]['caid']+'-points.txt', res)

In [ ]:
start_index = 0
start_id = target_2631.loc[start_index]['caid']
print("start index: %d, caid: %s" %(0, start_id))
end = 4724839
end_tmp = 472483
count = 0
is_first = True
res = 'LINESTRING('
for index in range(0, end_tmp):
  data_id = target_2631.loc[index]['caid']
  single_data = target_2631.loc[index]
  is_data_valid = (single_data['latitude'] < 38.85) & (single_data['latitude'] > 38.099) & (single_data['longitude'] < -122.35) & (single_data['longitude'] > -123.55)
  if data_id != start_id:
    # print("end index: %d, caid: %s" %(index - 1, target_2631.loc[index - 1]['caid']))
    # print("total points: %d" %(index - start_index))
    res+=')'
    if (index - start_index) > 49:
      write_file('data/GPS.nosync/points/' + str(count) +'-points.txt-tmp', res)
    is_first = True
    res = 'LINESTRING('
    count+=1
    start_index = index
    start_id = data_id
  lat = float(single_data['latitude'])
  lon = float(single_data['longitude'])
  #df.loc[len(df.index)] = [float(lat),float(lon)]
  if is_data_valid:
    if is_first:
      is_first = False
      res+= (str(lon) + ' ' + str(lat))
    else:
      res+= (',' + str(lon) + ' ' + str(lat))

    # print("start index: %d, caid: %s" %(index, data_id))
#print("end index: %d, caid: %s" %(end, target_2631.loc[end]['caid']))

In [ ]:
streets = gpd.read_file('data/GPS.nosync/Streets/Streets.shp')

In [ ]:
file_path1 = "data/GPS.nosync/output_test/sub-result-1603/1603-m_wkt.txt"
file_path2 = "data/GPS.nosync/output_test/sub-result-1603/1603-e_wkt.txt"
#check if file is present
if os.path.isfile(file_path1):
    #open text file in read mode
    text_file = open(file_path1, "r")
 
    #read whole file to a string
    wkt1 = text_file.read()
 
    #close file
    text_file.close()
if os.path.isfile(file_path2):
    #open text file in read mode
    text_file = open(file_path2, "r")
 
    #read whole file to a string
    wkt2 = text_file.read()
 
    #close file
    text_file.close()
d1 = {'col1': ['name1', 'name2'], 'wkt': wkt1}
d2 = {'col1': ['name1', 'name2'], 'wkt': wkt2}
df = pd.DataFrame(d1)
gs = gpd.GeoSeries.from_wkt(df['wkt'])
gdf1 = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")
df = pd.DataFrame(d2)
gs = gpd.GeoSeries.from_wkt(df['wkt'])
gdf2 = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 8))
streets.plot(ax=ax, color='black', LineWidth = 2)
gdf2.plot(ax=ax, color='green',LineWidth = 2)
#gdf.plot(ax=ax, color='blue', LineWidth = 1)
gdf1.plot(ax=ax, color='red',LineWidth = 1)

In [ ]:
def get_gdf_from_res(file_path):
  if os.path.isfile(file_path):
    #open text file in read mode
    text_file = open(file_path, "r")

    #read whole file to a string
    wkt = text_file.read()
    #close file
    text_file.close()
    if (wkt.count(',') + 1) > 4:
      d = {'col1': ['name1', 'name2'], 'wkt': wkt}
      df = pd.DataFrame(d)
      gs = gpd.GeoSeries.from_wkt(df['wkt'])
      gdf = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")
      gdf['points'] = gdf.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
      return gdf
    else:
      print('file_path %s contains so few points!' % file_path)
      return gpd.GeoDataFrame({'A' : []})
  else:
    return gpd.GeoDataFrame({'A' : []})


In [ ]:
30266 in df_new.index


In [ ]:
#from folium import plugins
#map = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12)
t1 = pd.Timestamp(year = 2019,  month = 10, day = 25,
           hour = 21, minute = 59, second = 59, tz = 'US/Pacific')
t2 = pd.Timestamp(year = 2019,  month = 10, day = 26,
           hour = 6, minute = 0, second = 0, tz = 'US/Pacific')
t3 = pd.Timestamp(year = 2019,  month = 10, day = 26,
           hour = 12, minute = 0, second = 0, tz = 'US/Pacific')
t4 = pd.Timestamp(year = 2019,  month = 10, day = 26,
           hour = 21, minute = 59, second = 59, tz = 'US/Pacific')
t5 = pd.Timestamp(year = 2019,  month = 10, day = 27,
           hour = 6, minute = 0, second = 0, tz = 'US/Pacific')
t6 = pd.Timestamp(year = 2019,  month = 10, day = 27,
           hour = 12, minute = 0, second = 0, tz = 'US/Pacific')
t7 = pd.Timestamp(year = 2019,  month = 10, day = 27,
           hour = 21, minute = 59, second = 59, tz = 'US/Pacific')
t8 = pd.Timestamp(year = 2019,  month = 10, day = 28,
           hour = 6, minute = 0, second = 0, tz = 'US/Pacific')
heat_data1 = [[0,0]]
heat_data2 = [[0,0]]
heat_data3 = [[0,0]]
heat_data4 = [[0,0]]
heat_data5 = [[0,0]]
heat_data6 = [[0,0]]
heat_data7 = [[0,0]]
t1 = pd.Timestamp(year = 2019,  month = 10, day = 27,
           hour = 3, minute = 20, second = 18, tz = 'US/Pacific')
for index in range(3078,30234):
  if index in df_new.index:
    time_flag = 0
    start_time = df_new.loc[index]['start_time']
    end_time = df_new.loc[index]['end_time']
    if (start_time >= t1) & (start_time < t2):
      time_flag = 1
    if (start_time >= t2) & (start_time < t3):
      time_flag = 2
    if (start_time >= t3) & (start_time < t4):
      time_flag = 3
    if (start_time >= t4) & (start_time < t5):
      time_flag = 4
    if (start_time >= t5) & (start_time < t6):
      time_flag = 5
    if (start_time >= t6) & (start_time < t7):
      time_flag = 6
    if (start_time >= t7) & (start_time < t8):
      time_flag = 7
    if time_flag > 0:
      file_path = 'data/GPS.nosync/output_test/sub-result-' + str(index) + '/' + str(index) + '-m_wkt.txt'
      gdf = get_gdf_from_res(file_path)
      
      if not gdf.empty:
        print('get the data of index:%d' % index)
        if time_flag == 1:
          for points in gdf['points']:
            for p in points:
                heat_data1.append([p[1],p[0]])
        if time_flag == 2:
          for points in gdf['points']:
            for p in points:
                heat_data2.append([p[1],p[0]])
        if time_flag == 3:
          for points in gdf['points']:
            for p in points:
                heat_data3.append([p[1],p[0]])
        if time_flag == 4:
          for points in gdf['points']:
            for p in points:
                heat_data4.append([p[1],p[0]])
        if time_flag == 5:
          for points in gdf['points']:
            for p in points:
                heat_data5.append([p[1],p[0]])
        if time_flag == 6:
          for points in gdf['points']:
            for p in points:
                heat_data6.append([p[1],p[0]])
        if time_flag == 7:
          for points in gdf['points']:
            for p in points:
                heat_data7.append([p[1],p[0]])
heat_data1 = heat_data1[1:]
heat_data2 = heat_data2[1:]
heat_data3 = heat_data3[1:]
heat_data4 = heat_data4[1:]
heat_data5 = heat_data5[1:]
heat_data6 = heat_data6[1:]
heat_data7 = heat_data7[1:]



In [ ]:
heat_data7

In [ ]:
import folium
from folium import plugins

map2 = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 14, min_zoom=10)

plugins.HeatMap(heat_data2,min_opacity=0.4,
                   radius=21, blur=12, 
                   max_zoom=4, min_zoom=8).add_to(map2)
#plugins.HeatMap(heat_data).add_to(map)
map2

In [ ]:
map3 = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 14, min_zoom=10)

plugins.HeatMap(heat_data3,min_opacity=0.4,
                   radius=21, blur=12, 
                   max_zoom=4, min_zoom=8).add_to(map3)
#plugins.HeatMap(heat_data).add_to(map)
map3


In [ ]:
map4 = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 14, min_zoom=10)

plugins.HeatMap(heat_data4,min_opacity=0.4,
                   radius=21, blur=12, 
                   max_zoom=4, min_zoom=8).add_to(map4)
#plugins.HeatMap(heat_data).add_to(map)
map4

In [ ]:
map5 = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 14, min_zoom=10)

plugins.HeatMap(heat_data3,min_opacity=0.4,
                   radius=21, blur=12, 
                   max_zoom=4, min_zoom=8).add_to(map5)
#plugins.HeatMap(heat_data).add_to(map)
map5

In [ ]:
map6 = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 14, min_zoom=10)

plugins.HeatMap(heat_data6,min_opacity=0.4,
                   radius=21, blur=12, 
                   max_zoom=4, min_zoom=8).add_to(map6)
#plugins.HeatMap(heat_data).add_to(map)
map6

In [ ]:
map7 = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 14, min_zoom=10)

plugins.HeatMap(heat_data7,min_opacity=0.4,
                   radius=21, blur=12, 
                   max_zoom=4, min_zoom=8).add_to(map7)
#plugins.HeatMap(heat_data).add_to(map)
map7

In [ ]:
map2.save("26-06-26-12.html")
map3.save("26-12-26-22.html")
map4.save("26-22-27-06.html")
map5.save("27-06-27-12.html")
map6.save("27-12-27-22.html")

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(64, 64))
streets.plot(ax=ax, color='black', LineWidth = 2)
gdf2.plot(ax=ax, color='green',LineWidth = 2)
#gdf.plot(ax=ax, color='blue', LineWidth = 1)
gdf1.plot(ax=ax, color='red',LineWidth = 1)

In [ ]:
# parts = streets[1:10000]
# parts['points'] = parts.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
# len(parts)
gdf2['points'] = gdf1.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
gdf1.head
gdf1

In [ ]:
import folium
map = folium.Map(location = [4,10], tiles = "Stamen Terrain", zoom_start = 3)

In [ ]:
from folium import plugins

map = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12)

#heat_data = [gdf2.points[0],gdf2.points[1]]
#heat_data[1] = 
heat_data = [[point[1], point[0]] for point in gdf2['points'][0] ]
# for point in gdf2['points'][1]:
#   heat_data.append([point[1],point[0]])
for points in gdf2['points']:
  for p in points:
    heat_data.append([p[1],p[0]])
heat_data
plugins.HeatMap(heat_data,min_opacity=0.5,
                   radius=4, blur=4, 
                   max_zoom=1).add_to(map)
map


In [ ]:
heat_data

In [ ]:
map.save("index-1-14-13-13.html")

In [ ]:
plugins.HeatMap(heat_data).add_to(map)
map